In [43]:
from selenium import webdriver
from time import sleep
import threading

sudo pip3 install opencv-python

In [27]:
%matplotlib inline

In [84]:
def get_codi(page, driver): 
    driver.get("http://www.mapssi.com/mapssi_web/codi.php")

    driver.execute_script('goPage({})'.format(page))
    sleep(0.5)

    lis = driver.find_elements_by_css_selector('.favorite_sub li') 
    
    idx = 0
    rows = []
    while(True):
        try:
            li = lis[idx]
            #link = article.find_element_by_css_selector('dt:not(.photo) > a').text
            link = li.find_element_by_css_selector('a').get_attribute("href")
            codino = li.find_element_by_css_selector('a').get_attribute('onclick')
            codino = codino.replace('javascript:goCodi(','').replace(')','').strip()
            
            tag = li.find_element_by_css_selector('a').text
            img = li.find_element_by_css_selector('a div.mc img:nth-child(2)').get_attribute('src')
            

            rank = (page-1) * 9 + idx + 1 
            rows.append([rank, codino, img, link, tag])
         
            # 종료
            idx = idx + 1
            if idx == len(lis):
                break
        except Exception as e:
            # 실패시 처음부터 다시 분석
            del rows[:]
            rows = [] 
            sleep(1) 
            
            lis = driver.find_elements_by_css_selector('.favorite_sub li')
            idx = 0 
            
            pass 
    return rows
    

def get_page(start, end):
    
    driver =  webdriver.PhantomJS()
    for page in range(start, end):
        codi_data = get_codi(page, driver)
        codi_all.extend(codi_data)
        
    driver.close() 


In [329]:
!killall phantomjs

No matching processes belonging to you were found


In [705]:
# 맵시 페이지 현재 165개의 페이지에 9개 아이템이 있다.
# 이를 스레드 16개를 이용하여 크롤링을 진행한다
# 총 코디 개수
9 * 11 * 15

1485

In [330]:
codi_all = []
for cnt in range(1, 16): 
    start = 11 * (cnt-1)+1
    end = 11 * cnt+1
    th = threading.Thread(target=get_page, args=(start, end))
    th.start()

In [ ]:
# 30초마다 개수 확인하여 크롤링 상황 체크
while True:
    tmp = len(codi_all)
    print(tmp)
    if 1485 == tmp:
        break
    sleep(30)

In [344]:
df = pd.DataFrame(codi_all, columns=['rank','id_codi','img','link','tag'])

In [346]:
df.to_csv('codi.csv', index=False)

In [709]:
# 중복 되는 코디 확인
len(df), len(set(df['id_codi']))

(1485, 1116)

In [644]:
def get_detail(codino, driver): 
    
    # 상세페이지 크롤링 시작
    # 코디넘버를 받아 클로링
    
    url = 'http://www.mapssi.com/mapssi_web/codi_detail.php?codiNo={}'.format(codino)
    driver.get(url)
    
    # 에러를 최소화 하기위해 인터블을 둔다.
    sleep(1)
    
    lis = driver.find_elements_by_css_selector('#codi_item_li li')
    
    rows = []
    try:
        for li in lis:
            item = li.find_element_by_css_selector('a').get_attribute('onclick')
            item = item.replace('javascript:goItem(', '')
            item = item.replace(')', '')
            iteminfo = item.split(',')
            itemno = iteminfo[0].strip()
            codino = iteminfo[1].strip()
            
            img = li.find_element_by_css_selector('span.img img:nth-child(2)').get_attribute('src')
            name = li.find_element_by_css_selector('.tit a').text
            brand = li.find_element_by_css_selector('p.brand').text
            price = li.find_element_by_css_selector('p.price').text
            price = price.replace('원', '')
            price = price.replace(',', '')
            
     
            rows.append([itemno, codino, name, brand, price, img])
        
    except Exception as e:
        print(codino, e)
        pass
     
    return rows

 
def get_detail_page(ids): 
    driver =  webdriver.PhantomJS() 
    for coid in ids:
        #print(coid)
        codi_data = get_detail(coid, driver)
        codi_all.extend(codi_data) 
        
    driver.close() 

In [711]:
# 확인해야 할 codi 1116개, 상세페이지 크롤링
# 스레드 6개를 이용하여 크롤링
1116 / 6

186.0

In [638]:
# 크롤링 할 고유 코디 넘버
unique_codino = list(set(df['id_codi'].values))

In [701]:
!killall phantomjs

In [646]:
codi_all = []
for cnt in range(1, 7): 
    start = 186 * (cnt-1)
    end = 186 * cnt
    ids = unique_codino[start : end]
    
    #print(start, end-1)
    #print(coid[0], coid[-1])
    #print('==') 
    #get_detail_page(ids)
    th = threading.Thread(target=get_detail_page, args=(ids,))
    th.start()

In [688]:
len(codi_all)

367

In [689]:
len(set(pd.DataFrame(codi_all)[1]))

64

In [690]:
# 1차 크롤링 완료
cdata = pd.DataFrame(codi_all, columns=['id_item','id_codi','name','brand','price','img'])
cdata.to_csv('items_1.csv', index=False)

In [686]:
# 1차 크롤링에서 누락된 코디넘버 체크하여 다시 크롤링 
ctid = list(set(cdata[1]))
ucid = list(set(df['id_codi']))
errors = []
for uci in ucid:
    if uci not in ctid:
        errors.append(uci)
        
codi_all = []
get_detail_page(errors)

In [ ]:
cdata = pd.DataFrame(codi_all, columns=['id_item','id_codi','name','brand','price','img'])
cdata.to_csv('items_2.csv', index=False)

In [693]:
# 1차 2차 크롤링 데이터 합치기
item1 = pd.read_csv('items_1.csv')
item2 = pd.read_csv('items_2.csv')
itemsdf = pd.concat([item1, item2])
itemsdf.to_csv('items.csv', index=False)